In [1]:
import pandas as pd
import numpy as np

In [3]:
df2010 = pd.read_csv('../Data/s2010.tdf', sep='\t', engine='python', names=['symb', 'retdate', 'opn', 'high', 'low','cls', 'vol', 'exch'], parse_dates = ['retdate'])
df2011 = pd.read_csv( '../Data/s2011.tdf',sep='\t', engine='python', names=['symb', 'retdate', 'opn', 'high', 'low','cls', 'vol', 'exch'], parse_dates = ['retdate'])
dffnd = pd.read_csv( '../Data/fnd.tdf',sep='\t', engine='python', names=['gvkey', 'datadate', 'fyear', 'indfmr','consol', 'popsrc', 'datafmt' , 'tic', 'cusip', 'conm', 'fyr', 'cash', 'dp','ebitda', 'emp', 'invt', 'netinc', 'ppent', 'rev' , 'ui', 'cik'])

In [4]:
df2010.dtypes

symb               object
retdate    datetime64[ns]
opn               float64
high              float64
low               float64
cls               float64
vol                 int64
exch               object
dtype: object

# First Five  

1. What is the total number of rows in the 2010 database? Return this as an integer.

In [9]:
len(df2010.index)

816066

2. How many unique symbols are there in 2010? Return this as an integer.

In [10]:
df2010.loc[:, 'symb'].nunique()

3362

3. What is the minimum closing price for a stock when it had a volume greater than 1,000,000 shares
in 2010?

In [18]:
df2010.loc[(df2010.loc[:, 'vol'] > 1000000), 'cls'].agg('min')

0.04

4. Return the average closing price for all stocks from the NYSE in 2010.

In [19]:
df2010.loc[(df2010.loc[:, 'exch'] == 'NYSE'), 'cls'].agg('mean')

94.89403912053864

5. Return the average closing price for all stocks and the total number of rows by the exchange of each
stock for 2010. Order the results from lowest to highest average closing price. This should return
two rows and three index/value columns (exchange, average closing price and total number of rows).

In [21]:
df2010.groupby('exch', as_index = False).agg({'cls': ['mean', 'count']})

exch          cls        
                  mean   count
0  NASDAQ  5971.784827  362220
1    NYSE    94.894039  453846

# Main Problems

1. Which symbols have less than 50 rows in 2010?

In [28]:
temp = df2010.copy().groupby('symb').agg({'symb': ['count']}).reset_index()
temp.columns = ['symb', 'nrows']
temp.loc[(temp.loc[:, 'nrows'] < 50), :]

,symb,nrows
35,ACNB,8
80,AG,12
306,BAH,31
354,BEP,19
390,BITA,31
442,BOX,45
527,CAR,1
564,CCCL,41
688,CMRE,40
712,CODI,43


2. How many symbols have less than 50 rows in 2010?

In [30]:
temp = df2010.copy().groupby('symb').agg({'symb': ['count']}).reset_index()
temp.columns = ['symb', 'nrows']
temp.loc[(temp.loc[:, 'nrows'] < 50), 'symb'].count()

53

3. Write a query which returns one row and two columns (DataFrame or Series). The first column
should contain the number of symbols which have less than 50 rows in 2010 and the second column
should have the number of symbols with more than 100 rows in 2010.

In [33]:
temp = df2010.copy().groupby('symb').agg({'symb': ['count']}).reset_index()
temp.columns = ['symb', 'nrows']
out = pd.DataFrame({'lessThan50': temp.loc[(temp.loc[:, 'nrows'] < 50), 'symb'].count(), 'moreThan100': temp.loc[(temp.loc[:, 'nrows'] > 100), 'symb'].count()}, index = [1])
out

,lessThan50,moreThan100
1,53,3266


4. Write a query which returns two column and two rows (either series or dataFrame). The first column
should be equal to “lessThan50” or “moreThan100” and the second column should have the number
of unique symbols which correspond to this condition. In other words, the same numbers as the
previous problem, transposed with an column providing a description.

In [34]:
temp = df2010.copy().groupby('symb').agg({'symb': ['count']}).reset_index()
temp.columns = ['symb', 'nrows']
temp.loc[:, 'flag'] = 'not_interested'
temp.loc[(temp.loc[:, 'nrows'] < 50), 'flag'] = 'lessThan50'
temp.loc[(temp.loc[:, 'nrows'] > 100), 'flag'] = 'moreThan100'
temp.loc[(temp.loc[:, 'flag'] != 'not_interested'), :].groupby('flag', as_index = False).agg({'nrows': ['count']})

,flag,nrows
,,count
0,lessThan50,53
1,moreThan100,3266


5. Write a query which returns three rows and two columns (note that one column maybe an index).
One column should contain the average yearly total traded volume for symbols which had (1) more
than 100 trading days (2) less than 50 trading days and (3) between 50 and 100 trading days. The
other column should identify each row and be called “numType".

In [43]:
temp = df2010.copy().groupby('symb').agg({'symb': ['count'], 'vol': ['sum']}).reset_index()
temp.columns = ['symb', 'nrows', 'tot_vol']
temp.loc[:, 'numType'] = 'between'
temp.loc[(temp.loc[:, 'nrows'] < 50), 'numType'] = 'lessThan50'
temp.loc[(temp.loc[:, 'nrows'] > 100), 'numType'] = 'moreThan100'
temp.groupby('numType', as_index = False).agg({'tot_vol': ['mean']})

,numType,tot_vol
,,mean
0,between,2.416473e+07
1,lessThan50,3.552224e+07
2,moreThan100,3.736511e+08


6. Write a query which returns three rows and two columns. The first column should contain the average
daily traded volume for symbols which had (1) more than 100 trading days (2) less than 50 trading
days and (3) between 50 and 100 trading days. The other column should identify each row and be
called “numType.”

In [44]:
temp = df2010.copy().groupby('symb').agg({'symb': ['count'], 'vol': ['mean']}).reset_index()
temp.columns = ['symb', 'nrows', 'avg_vol']
temp.loc[:, 'numType'] = 'between'
temp.loc[(temp.loc[:, 'nrows'] < 50), 'numType'] = 'lessThan50'
temp.loc[(temp.loc[:, 'nrows'] > 100), 'numType'] = 'moreThan100'
temp.groupby('numType', as_index = False).agg({'avg_vol': ['mean']})

,numType,avg_vol
,,mean
0,between,3.502071e+05
1,lessThan50,1.268627e+06
2,moreThan100,1.487666e+06


7. How many of the symbols had a day where the dollar volume (closing price multiplied by number of
shares traded) was greater than 100 million dollars in 2010?

In [50]:
df2010.loc[(df2010.loc[:, 'cls']*df2010.loc[:, 'vol'] > 100000000), 'symb'].nunique()

964

8. What percentage of the symbols had a day where the dollar volume of shares traded was greater
than 100 million dollars in 2010?

In [51]:
(df2010.loc[(df2010.loc[:, 'cls']*df2010.loc[:, 'vol'] > 100000000), 'symb'].nunique())/(df2010.loc[:, 'symb'].nunique())

0.2867340868530637